In [147]:
import pandas as pd
from backtesting.test import EURUSD, SMA
import backtesting
import backtesting.lib as btlib

backtesting.set_bokeh_output(notebook=False)

data = EURUSD.iloc[:500, :].copy()
# 重采样日周期数据
data_daily = data.resample('D', label='right').agg(btlib.OHLCV_AGG).dropna()
print(data.info(), data.head())
print(data_daily.info(), data_daily.head())



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 500 entries, 2017-04-19 09:00:00 to 2017-05-18 04:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    500 non-null    float64
 1   High    500 non-null    float64
 2   Low     500 non-null    float64
 3   Close   500 non-null    float64
 4   Volume  500 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 23.4 KB
None                         Open     High      Low    Close  Volume
2017-04-19 09:00:00  1.07160  1.07220  1.07083  1.07219    1413
2017-04-19 10:00:00  1.07214  1.07296  1.07214  1.07260    1241
2017-04-19 11:00:00  1.07256  1.07299  1.07170  1.07192    1025
2017-04-19 12:00:00  1.07195  1.07280  1.07195  1.07202    1460
2017-04-19 13:00:00  1.07200  1.07230  1.07045  1.07050    1554
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26 entries, 2017-04-20 to 2017-05-19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
-

In [160]:

from backtesting import Strategy, Backtest
from backtesting import backtesting
from backtesting.lib import resample_apply


class System(Strategy):

    def log(self, txt, dt=None):
        """ 此策略的日志功能"""
        dt = dt or self.data.index[-1]
        print('%s, %s' % (dt, txt))

    def init(self):
        # 让我们将收盘价转换回熊猫系列。
        close = self.data.Close.s
        # 重新采样为每日分辨率。使用最后一个值（即一天结束时的收盘价）聚合组。
        # 注意`label='right'`。如果将其设置为“左”（默认），则该策略将表现出前瞻性偏差。
        # close_daily = close.resample('D', label='right').agg('last')
        # data_daily = self.data.df.resample('D', label='right').agg(btlib.OHLCV_AGG).dropna()

        # 添加指标昨日最高点
        self.d_high = resample_apply('D', lambda x: x, self.data.High, name='d_high')
        # 添加指标昨日最低点
        self.d_low = resample_apply('D', lambda x: x, self.data.Low, name='d_low')

        # self.signal_buy = (pd.Series(self.data.Close) > self.d_high).astype(int).diff().fillna(0)
        # self.signal_sell = (pd.Series(self.data.Close) < self.d_low).astype(int).diff().fillna(0)
        # self.signal = self.signal_buy + self.signal_sell
        print(self.data.index[0])
        print(self.data.index[-1])
        print()

    def next(self):
        price = self.data.Close


        signal_buy = price[-2] < self.d_high[-1] < price[-1]
        signal_sell = price[-2] > self.d_low[-1] > price[-1]
        # 如果我们还没有持仓，并且满足所有条件，则输入多头。
        if signal_buy:
            if self.position.is_short:
                self.position.close()
        if signal_buy and not self.position.is_long:
        # if btlib.cross(price, self.d_high):
            # 下次开盘时以市价买入。
            self.buy(size=.1)
            self.log("buy %f" % price[-1])

        elif signal_sell:
            if self.position.is_long:
                self.position.close()
        if signal_sell and not self.position.is_short:
        # elif btlib.cross(price, self.d_low):
            self.sell(size=.1)
            self.log("sell %f %s" % (price[-1], self.position.size))



backtest = Backtest(data, System, commission=.0002, margin=.05)
backtest.run()

# backtest.plot()

2017-04-19 09:00:00
2017-05-18 04:00:00

2017-04-20 06:00:00, buy 1.074140
2017-04-21 09:00:00, sell 1.070530 18616
2017-04-23 21:00:00, buy 1.089800
2017-04-24 00:00:00, sell 1.085040 17557
2017-04-25 06:00:00, buy 1.088270
2017-05-01 04:00:00, sell 1.089080 17314
2017-05-01 12:00:00, buy 1.091690
2017-05-03 20:00:00, sell 1.088520 17186


KeyboardInterrupt: 